# Advanced Data Analysis Exercise


*"... This dataset contains average counts of bicycle traffic by hour of the day and day of the week. Data is collected from bicycle counters across approximately 16 active transport sites. As more sites come on line in the future these will be added to the dataset..."* - from [Queensland Government Open Data Portal](https://www.data.qld.gov.au/dataset/average-bicycle-counts-by-day-and-hour/resource/0234d8d0-b1bf-485f-877a-cc7524db4d5a)

**Replace "__" in the following codes with a proper value**

## Load Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('averagebicyclecounts-bydayandhourfromjanuary2020-tojune2020.csv')
data

## Find the busiest day with the most bicycles on the road

### Clean the dataset
- Change the table from the wide format to the tall format
- Drop and Rename the table columns

In [ ]:
# convert from wide to tall table format
tall_df = data.melt(id_vars=["SITE_ID", "DESCRIPTION", "YEARS", "MONTHS", "HOURS"], 
                    var_name="Days",
                    value_name="BicycleCount")

# convert "Days" column from string to a categorical variable
tall_df["Days"] = pd.Categorical(tall_df["Days"], 
                                categories=["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"], 
                                ordered=True)
tall_df.head()

### Examine the average bicycle count on different days of the week

In [ ]:
daily_bicycle_count = tall_df \
    .groupby(["__"])["__"] \
    .mean()
daily_bicycle_count

In [ ]:
average_bicycle_count = daily_bicycle_count.mean()
print(f"On average, there are {average_bicycle_count:.2f} bicycles daily on the road across all the sites.")

busiest_day = daily_bicycle_count.idxmax()
print(f"The busiest day is {busiest_day} (average {daily_bicycle_count[busiest_day]:.2f} bicycles)")

In [ ]:
daily_bicycle_count \
    .reset_index() \
    .plot("__", "__", 
          kind="bar", 
          style=".-",
          figsize=(8,6))
plt.grid()
plt.xlabel("Days")
plt.ylabel("Bicycle Count")
plt.title("Daily Total Bicycle Count")

**Observations**:
- Everyday, there are 23.07 bicycles on the road across all the sites in Queensland.
- Queensland's people tends to ride bicycle more often during weekends.
- They also like to ride their bicycles on Tuesday, with 1.3 more bicycles than the daily average.


## Examine the average daily bicycle count per site in Queensland

In [ ]:
# Group the data by the site and days
# Check the average bicycle count by site
avg_bicycle_count_per_site = tall_df \
    .groupby(["__", "__"])["__"] \
    .mean() \
    .reset_index()

avg_bicycle_count_per_site

## Find the top 10 busiest site with the highest daily average of bicycle count

In [ ]:
# Find the top 10 site with the most daily average of bicycle count
# Replace the "__" with a proper variable name

top_10_sites = avg_bicycle_count_per_site \
    .groupby("__")["__"] \
    .mean() \
    .sort_values(ascending=False) \
    .head(10) \
    .reset_index()
    
top_10_sites

In [ ]:
top_10_sites.plot("__", "__", kind="bar")
plt.ylabel("Bicycle Count")
plt.title("Top 10 Sites")

**Observations**
- Park Road, Woolloongabba has the highest daily average of bicycle on the road. The number of bicycles is three times higher than the daily average across all the sites

### Food for thoughts
Assume that you are a government officer responsible for the road planning, 
1. Which roads should be constantly monitored to ensure the safety of the cyclists? Or even invested to ensure it can handle a large amount of cyclists?

1. Is the results in this analysis sufficient for used in road planning? What can we do better to inform the public and government? 